In [1]:
import pandas as pd
import numpy as np
import findspark
findspark.init()

In [2]:
import pyspark

create pyspark context

In [3]:
sc=pyspark.SparkContext(appName='joao')

22/07/03 09:23:54 WARN Utils: Your hostname, LT0093 resolves to a loopback address: 127.0.1.1; using 172.23.76.255 instead (on interface eth0)
22/07/03 09:23:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/03 09:23:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


resilient distributed dataset

In [4]:
rdd=sc.parallelize([i for i in range(100)])
rdd_squared=rdd.map(lambda x: x**2)

In [ ]:
rdd_squared.collect()

## SQL DataFrame

In [6]:
from sklearn.datasets import load_breast_cancer

In [7]:
# create sql session
sql_session=pyspark.sql.SparkSession(sc)

In [8]:
d=load_breast_cancer()
df=pd.DataFrame(d['data'])
df['target']=d['target']

In [9]:
df_spark=sql_session.createDataFrame(df)

In [10]:
type(df_spark)

pyspark.sql.dataframe.DataFrame

In [14]:
df_spark.count()

569

In [15]:
df_spark.show(5)

+-----+-----+-----+------+-------+-------+------+-------+------+-------+------+------+-----+-----+--------+-------+-------+-------+-------+--------+-----+-----+-----+------+------+------+------+------+------+-------+------+
|    0|    1|    2|     3|      4|      5|     6|      7|     8|      9|    10|    11|   12|   13|      14|     15|     16|     17|     18|      19|   20|   21|   22|    23|    24|    25|    26|    27|    28|     29|target|
+-----+-----+-----+------+-------+-------+------+-------+------+-------+------+------+-----+-----+--------+-------+-------+-------+-------+--------+-----+-----+-----+------+------+------+------+------+------+-------+------+
|17.99|10.38|122.8|1001.0| 0.1184| 0.2776|0.3001| 0.1471|0.2419|0.07871| 1.095|0.9053|8.589|153.4|0.006399|0.04904|0.05373|0.01587|0.03003|0.006193|25.38|17.33|184.6|2019.0|0.1622|0.6656|0.7119|0.2654|0.4601| 0.1189|     0|
|20.57|17.77|132.9|1326.0|0.08474|0.07864|0.0869|0.07017|0.1812|0.05667|0.5435|0.7339|3.398|74.08|0.0052

In [16]:
df_spark.printSchema()

root
 |-- 0: double (nullable = true)
 |-- 1: double (nullable = true)
 |-- 2: double (nullable = true)
 |-- 3: double (nullable = true)
 |-- 4: double (nullable = true)
 |-- 5: double (nullable = true)
 |-- 6: double (nullable = true)
 |-- 7: double (nullable = true)
 |-- 8: double (nullable = true)
 |-- 9: double (nullable = true)
 |-- 10: double (nullable = true)
 |-- 11: double (nullable = true)
 |-- 12: double (nullable = true)
 |-- 13: double (nullable = true)
 |-- 14: double (nullable = true)
 |-- 15: double (nullable = true)
 |-- 16: double (nullable = true)
 |-- 17: double (nullable = true)
 |-- 18: double (nullable = true)
 |-- 19: double (nullable = true)
 |-- 20: double (nullable = true)
 |-- 21: double (nullable = true)
 |-- 22: double (nullable = true)
 |-- 23: double (nullable = true)
 |-- 24: double (nullable = true)
 |-- 25: double (nullable = true)
 |-- 26: double (nullable = true)
 |-- 27: double (nullable = true)
 |-- 28: double (nullable = true)
 |-- 29: double (nu

In [18]:
df_spark.columns

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 'target']

In [20]:
col_0= df_spark.select('0')

In [22]:
col_0.columns

['0']

In [28]:
col_0.agg({'0':'sum'}).collect()[0]

Row(sum(0)=8038.429000000001)

In [39]:
df_spark.describe().toPandas()

,summary,0,1,2,3,4,5,6,7,8,...,21,22,23,24,25,26,27,28,29,target
0,count,569,569,569,569,569,569,569,569,569,...,569,569,569,569,569,569,569,569,569,569
1,mean,14.127291739894554,19.289648506151146,91.96903339191563,654.8891036906855,0.09636028119507907,0.10434098418277679,0.0887993158172232,0.04891914586994728,0.18116186291739897,...,25.677223198594024,107.26121265377857,880.5831282952548,0.1323685940246046,0.25426504393673116,0.27218848330404216,0.11460622319859401,0.2900755711775044,0.08394581722319859,0.6274165202108963
2,stddev,3.5240488262120775,4.301035768166949,24.298981038754906,351.914129181653,0.01406412813767362,0.052812757932512194,0.07971980870789348,0.038802844859153605,0.02741428133603572,...,6.146257623038319,33.602542269036356,569.3569926699489,0.02283242940483546,0.157336488913742,0.2086242806081323,0.06573234119594205,0.06186746753751869,0.01806126734889399,0.4839179564031687
3,min,6.981,9.71,43.79,143.5,0.05263,0.01938,0.0,0.0,0.106,...,12.02,50.41,185.2,0.07117,0.02729,0.0,0.0,0.1565,0.05504,0
4,max,28.11,39.28,188.5,2501.0,0.1634,0.3454,0.4268,0.2012,0.304,...,49.54,251.2,4254.0,0.2226,1.058,1.252,0.291,0.6638,0.2075,1


to RDD

In [50]:
df_spark.rdd.collect()[0:1]

[Row(0=17.99, 1=10.38, 2=122.8, 3=1001.0, 4=0.1184, 5=0.2776, 6=0.3001, 7=0.1471, 8=0.2419, 9=0.07871, 10=1.095, 11=0.9053, 12=8.589, 13=153.4, 14=0.006399, 15=0.04904, 16=0.05373, 17=0.01587, 18=0.03003, 19=0.006193, 20=25.38, 21=17.33, 22=184.6, 23=2019.0, 24=0.1622, 25=0.6656, 26=0.7119, 27=0.2654, 28=0.4601, 29=0.1189, target=0)]

In [63]:
a=[("a", 1), ("b", 4)]
b=[("a", 2), ("a", 3)]
a=sc.parallelize(a)
b=sc.parallelize(b)

In [67]:
a=a.map(lambda x: (x[0],x[1]+10))

In [70]:
c=a.join(b)

In [72]:
c.collect()

[('a', (11, 2)), ('a', (11, 3))]

22/07/03 12:31:00 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2050750 ms exceeds timeout 120000 ms
22/07/03 12:31:00 WARN SparkContext: Killing executors is not supported by current scheduler.
22/07/04 07:21:43 WARN NettyRpcEnv: Ignored failure: java.util.concurrent.TimeoutException: Cannot receive any reply from 172.23.76.255:41197 in 10000 milliseconds


## SQL Query

In [40]:
df_spark.createOrReplaceTempView('table_1')

In [43]:
type(sql_session)

pyspark.sql.session.SparkSession

In [42]:
sql_session.sql('select * from table_1 limit 2').show()

+-----+-----+-----+------+-------+-------+------+-------+------+-------+------+------+-----+-----+--------+-------+-------+-------+-------+--------+-----+-----+-----+------+------+------+------+------+------+-------+------+
|    0|    1|    2|     3|      4|      5|     6|      7|     8|      9|    10|    11|   12|   13|      14|     15|     16|     17|     18|      19|   20|   21|   22|    23|    24|    25|    26|    27|    28|     29|target|
+-----+-----+-----+------+-------+-------+------+-------+------+-------+------+------+-----+-----+--------+-------+-------+-------+-------+--------+-----+-----+-----+------+------+------+------+------+------+-------+------+
|17.99|10.38|122.8|1001.0| 0.1184| 0.2776|0.3001| 0.1471|0.2419|0.07871| 1.095|0.9053|8.589|153.4|0.006399|0.04904|0.05373|0.01587|0.03003|0.006193|25.38|17.33|184.6|2019.0|0.1622|0.6656|0.7119|0.2654|0.4601| 0.1189|     0|
|20.57|17.77|132.9|1326.0|0.08474|0.07864|0.0869|0.07017|0.1812|0.05667|0.5435|0.7339|3.398|74.08|0.0052

## Machine Learning

In [53]:
from pyspark.mllib.classification import LogisticRegressionWithSGD, LogisticRegressionWithLBFGS
from pyspark.mllib.regression import LabeledPoint

In [ ]:
from pyspark.mllib.clustering import KMeans

In [61]:
data=[]
for row, y in zip(d['data'],d['target']):
    p=LabeledPoint(y,row)
    data.append(p)

In [62]:
data=sc.parallelize(data)

In [63]:
d.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

In [64]:
lrm=LogisticRegressionWithSGD.train(data,iterations=100)

/Users/JoaoGomes/miniconda3/envs/spark/lib/python3.9/site-packages/pyspark/mllib/classification.py:395: FutureWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  warnings.warn(


In [66]:
lrm.weights

DenseVector([21.4597, 39.6495, 131.1845, 169.2285, 0.2244, 0.073, -0.1287, -0.0651, 0.4243, 0.1678, 0.1258, 3.1275, 0.6994, -43.6572, 0.0191, 0.0305, 0.0299, 0.0138, 0.0525, 0.0083, 20.6254, 50.6104, 125.4718, -166.4565, 0.2939, 0.0845, -0.1652, -0.0265, 0.6148, 0.1871])